In [1]:
from sqlalchemy.orm import registry

mapper_registry = registry()
Base = mapper_registry.generate_base()

from sqlalchemy import select
from sqlalchemy import Table, Column, Integer, String
from sqlalchemy import ForeignKey

from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session

from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)


class User(Base):
    __tablename__ = "user_account"
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return (
            f"User(id={self.id!r},"
            f" name={self.name!r},"
            f" fullname={self.fullname!r})"
        )


class Address(Base):
    __tablename__ = "address"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("user_account.id"))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address(id={self.id!r}," f" email_address=" f"{self.email_address!r})"

2022-04-03 17:59:32,422 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 17:59:32,423 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-03 17:59:32,424 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:59:32,424 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-04-03 17:59:32,425 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:59:32,427 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-03 17:59:32,427 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:59:32,428 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-04-03 17:59:32,429 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 17:59:32,430 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-03 17:59:32,431 INFO sqlalchemy.engine.Engine [no key 0.00064s] ()
2022-04-03 17:59:32,432 INFO sqlalchemy.engine.Engine 
C

In [ ]:
mapper_registry.metadata.create_all(engine)

In [8]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)
metadata_obj.create_all(engine)


2022-04-03 18:02:31,523 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:02:31,525 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-03 18:02:31,526 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:02:31,527 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-03 18:02:31,528 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-03 18:02:31,531 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
from sqlalchemy import insert
stmt = insert(user_table).\
    values(name='spongebob', 
    fullname="Spongebob Squarepants")

In [6]:
from sqlalchemy import select, bindparam

scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {"username": "spongebob", "email_address": "spongebob@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2022-04-03 18:00:46,141 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:00:46,142 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2022-04-03 18:00:46,143 INFO sqlalchemy.engine.Engine [generated in 0.00231s] (('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org'))
2022-04-03 18:00:46,144 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
stmt = select(User).where(User.name == 'spongebob')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-04-03 18:00:50,387 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:00:50,390 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-03 18:00:50,392 INFO sqlalchemy.engine.Engine [cached since 35.49s ago] ('spongebob',)
2022-04-03 18:00:50,393 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"


class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"

C:\Users\18463\AppData\Local\Temp\ipykernel_17708\2657691811.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  class User(Base):
C:\Users\18463\AppData\Local\Temp\ipykernel_17708\2657691811.py:10: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Address, and will be replaced in the string-lookup table.
  class Address(Base):


In [11]:
stmt = select(User).where(User.name == 'spongebob')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-04-03 18:02:53,028 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-03 18:02:53,031 INFO sqlalchemy.engine.Engine ROLLBACK


InvalidRequestError: Multiple classes found for path "Address" in the registry of this declarative base. Please use a fully module-qualified path.

In [13]:
import sqlalchemy
sqlalchemy.__version__

'1.4.32'